In [1]:
# importando librerias
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import collections
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost
import math as mt
import warnings

# Ploting styles
# styles: 'fivethirtyeight', 'classic', 'ggplot', 'seaborn-notebook'
# styles: 'seaborn-poster', 'bmh', 'grayscale', 'seaborn-whitegrid'
matplotlib.style.use('bmh')
%matplotlib inline
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#print(plt.style.available)
warnings.filterwarnings("ignore")

In [2]:
#propios utilitarios
from utils import *
data_train = pd.read_csv('data/01dataBaseTrainTrxRec.csv', **set_parameter_csv)
data_train.head()

,fechaOper,codCliente,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab,ctdTrx,ratingMonto
0,2017-01-29 00:00:00,7649,138.0,43629,1,176.0,1,0.014072
1,2016-12-01 00:00:00,24604,75.0,4326,0,81.0,1,0.001667
2,2017-06-01 00:00:00,15289,75.0,4326,0,81.0,1,0.000127
3,2017-09-22 00:00:00,5190,110.0,59776,1,156.0,1,0.001167
4,2017-05-05 00:00:00,16635,75.0,31043,0,81.0,1,0.047386


In [3]:
data_train.sort_values(by=['codCliente','codEstab']).head()

,fechaOper,codCliente,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab,ctdTrx,ratingMonto
25041,2017-05-24 00:00:00,1,105.0,39,1,147.0,1,0.084590
522851,2017-03-07 00:00:00,1,105.0,39,1,147.0,1,0.098481
743922,2017-08-20 00:00:00,1,105.0,39,1,147.0,1,0.018126
1272804,2017-05-13 00:00:00,1,105.0,39,1,147.0,1,0.018183
816568,2016-12-22 00:00:00,1,72.0,1194,1,147.0,1,0.031735


In [4]:
pd.concat([
    pd.DataFrame(data_train.groupby(['codCliente'])['ratingMonto'].sum()),
    pd.DataFrame(data_train.groupby(['codCliente'])['ratingMonto'].mean()),
    pd.DataFrame(data_train.groupby(['codCliente'])['ratingMonto'].count())],
    axis=1
)[:5]

,ratingMonto,ratingMonto,ratingMonto
codCliente,,,
1,0.863403,0.023983,36
2,0.540837,0.004545,119
3,0.582373,0.044798,13
4,0.774747,0.004280,181
5,0.621249,0.015152,41


In [5]:
pd.concat([
    pd.DataFrame(data_train.groupby(['codCliente', 'codEstab'])['ratingMonto'].sum()),
    pd.DataFrame(data_train.groupby(['codCliente', 'codEstab'])['ratingMonto'].mean()),
    pd.DataFrame(data_train.groupby(['codCliente', 'codEstab'])['ratingMonto'].count())],
    axis=1
)[:5]

ratingMonto  ratingMonto  ratingMonto
codCliente codEstab                                       
1          39           0.219380     0.054845            4
           1194         0.031735     0.031735            1
           1911         0.119995     0.119995            1
           2399         0.009543     0.009543            1
           2425         0.024790     0.012395            2

In [6]:
data_perfil = pd.read_csv('data/02dataBasePerfilRec.csv', **set_parameter_csv)
data_perfil.head()

,codCliente,rangoEdad,rangoIngreso,flagGenero,flagLimaProvCliente,ubigeoCliente,rangoCtdProdAct,rangoCtdProdPas,rangoCtdProdSeg,flagBxi,saldoTcEntidad1,saldoTcEntidad2,saldoTcEntidad3,saldoTcEntidad4
0,24041,Rango2,Rango2,1,0,360.0,Rango1,Rango1,Rango1,1,NaN,NaN,NaN,NaN
1,9040,Rango4,Rango6,1,1,260.0,Rango1,Rango1,Rango1,1,Rango1,Rango1,Rango1,Rango1
2,18233,Rango2,Rango1,1,1,273.0,Rango1,Rango2,Rango1,1,Rango2,Rango1,Rango1,Rango1
3,7226,Rango5,Rango3,0,1,241.0,Rango1,Rango2,Rango1,1,Rango1,Rango2,Rango1,Rango2
4,21617,Rango3,Rango2,1,1,268.0,Rango1,Rango1,Rango1,0,NaN,NaN,NaN,NaN


In [7]:
data_perfil.sort_values(by=['codCliente']).head()

,codCliente,rangoEdad,rangoIngreso,flagGenero,flagLimaProvCliente,ubigeoCliente,rangoCtdProdAct,rangoCtdProdPas,rangoCtdProdSeg,flagBxi,saldoTcEntidad1,saldoTcEntidad2,saldoTcEntidad3,saldoTcEntidad4
1599,1,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1
25805,2,Rango6,Rango2,1,1,241.0,Rango1,Rango3,Rango1,0,Rango6,Rango1,Rango1,Rango1
700,3,Rango6,Rango5,0,1,253.0,Rango1,Rango2,Rango2,0,Rango6,Rango2,Rango1,Rango2
19987,4,Rango6,Rango1,1,0,212.0,Rango1,Rango1,Rango2,0,Rango1,Rango1,Rango1,Rango1
29861,5,Rango6,Rango1,1,1,271.0,Rango1,Rango1,Rango3,1,Rango2,Rango1,Rango5,Rango1


In [8]:
data_train.shape, data_perfil.shape, data_train['codCliente'].unique().shape

((1591617, 8), (30000, 14), (30000,))

In [9]:
TrainTot = pd.merge(data_train, data_perfil, on='codCliente')
TrainTot.head()

,fechaOper,codCliente,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab,ctdTrx,ratingMonto,rangoEdad,rangoIngreso,flagGenero,flagLimaProvCliente,ubigeoCliente,rangoCtdProdAct,rangoCtdProdPas,rangoCtdProdSeg,flagBxi,saldoTcEntidad1,saldoTcEntidad2,saldoTcEntidad3,saldoTcEntidad4
0,2017-01-29 00:00:00,7649,138.0,43629,1,176.0,1,0.014072,Rango4,Rango2,1,1,281.0,Rango1,Rango2,Rango1,1,Rango3,Rango1,Rango1,Rango1
1,2017-05-07 00:00:00,7649,75.0,830,1,157.0,1,0.000821,Rango4,Rango2,1,1,281.0,Rango1,Rango2,Rango1,1,Rango3,Rango1,Rango1,Rango1
2,2016-12-20 00:00:00,7649,75.0,829,1,175.0,1,0.001220,Rango4,Rango2,1,1,281.0,Rango1,Rango2,Rango1,1,Rango3,Rango1,Rango1,Rango1
3,2017-02-16 00:00:00,7649,114.0,21603,1,175.0,1,0.000985,Rango4,Rango2,1,1,281.0,Rango1,Rango2,Rango1,1,Rango3,Rango1,Rango1,Rango1
4,2017-08-30 00:00:00,7649,110.0,46216,1,203.0,1,0.003635,Rango4,Rango2,1,1,281.0,Rango1,Rango2,Rango1,1,Rango3,Rango1,Rango1,Rango1


In [10]:
# Eliminacion de columnas fecha y Ctdrx
TrainTot.drop(['fechaOper', 'ctdTrx'], axis=1, inplace=True)
TrainTot.shape, TrainTot['codCliente'].unique().shape, TrainTot.columns

((1591617, 19),
 (30000,),
 Index(['codCliente', 'codGiro', 'codEstab', 'flagLimaProvEstab', 'ubigeoEstab',
        'ratingMonto', 'rangoEdad', 'rangoIngreso', 'flagGenero',
        'flagLimaProvCliente', 'ubigeoCliente', 'rangoCtdProdAct',
        'rangoCtdProdPas', 'rangoCtdProdSeg', 'flagBxi', 'saldoTcEntidad1',
        'saldoTcEntidad2', 'saldoTcEntidad3', 'saldoTcEntidad4'],
       dtype='object'))

In [11]:
TrainTot['ratingMonto'].describe()

count    1.591617e+06
mean     1.318383e-02
std      3.236438e-02
min      2.984905e-06
25%      1.598366e-03
50%      4.199563e-03
75%      1.152572e-02
max      1.000000e+00
Name: ratingMonto, dtype: float64

In [12]:
#Carga de CSV a entregar soluciones
BaseSub = pd.read_csv('data/03dataBaseTestRec.csv')
BaseSub.head()

,codClienteCodEstab
0,100001675
1,1000217838
2,1000219843
3,1000231391
4,100032061


In [13]:
BasePred = pd.read_csv('data/05dataBaseTestKeyRec.csv')
BasePred.sort_values(by=['codCliente', 'codEstab']).head()

,codCliente,codEstab
28161,1,39
244120,1,3843
40138,1,6314
407005,1,12767
378968,1,26260


In [14]:
BaseSub.shape, BasePred.shape, BaseSub['codClienteCodEstab'].unique().shape

((467203, 1), (467203, 2), (467203,))

In [15]:
print(data_train.shape)
combine_client_estab = list((data_train["codCliente"].map(str) + data_train["codEstab"].map(str)).unique())
print(len(combine_client_estab))
set_client_estab = set(combine_client_estab)
print(len(set_client_estab))

(1591617, 8)
901865
901865


In [16]:
set_est_client_pred = set(list(BaseSub['codClienteCodEstab'].map(str).unique()))
len(set_est_client_pred)

467203

In [17]:
len(set_client_estab & set_est_client_pred), len(set_client_estab | set_est_client_pred)

(194237, 1174831)

In [18]:
len(set_est_client_pred - set_client_estab)

272966

## Analisis de su eliminacion de duplicados

In [19]:
FeatEstab = data_train.drop(['codCliente','ratingMonto', 'fechaOper', 'ctdTrx'], axis=1)
print(FeatEstab.shape, FeatEstab.columns)
FeatEstab.sort_values(by=['codEstab']).head()

(1591617, 4) Index(['codGiro', 'codEstab', 'flagLimaProvEstab', 'ubigeoEstab'], dtype='object')


,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab
568892,NaN,1,0,NaN
636060,NaN,1,0,NaN
1398343,NaN,1,0,NaN
398126,NaN,1,0,NaN
808662,NaN,1,0,NaN


In [20]:
# eliminando duplicados
FeatEstab = FeatEstab.drop_duplicates()
print(FeatEstab.shape)
FeatEstab.sort_values(by=['codEstab']).head()

(74339, 4)


,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab
398126,NaN,1,0,NaN
9507,NaN,2,0,NaN
142715,100.0,3,1,166.0
568696,100.0,4,1,401.0
1250161,100.0,5,1,175.0


In [21]:
len(set(list(BasePred.codCliente)) & set(list(data_perfil.codCliente))), BasePred.codCliente.unique().shape, data_perfil.codCliente.unique().shape

(29905, (29905,), (30000,))

In [22]:
BasePred.shape, data_perfil.shape

((467203, 2), (30000, 14))

In [23]:
TesTot = pd.merge(BasePred, data_perfil, how='left', on=['codCliente'])
TesTot.shape

(467203, 15)

In [24]:
TesTot = pd.merge(TesTot, FeatEstab, how='left', on=['codEstab'])
TesTot.shape

(467203, 18)

In [26]:
TesTot.dtypes

codCliente               int64
codEstab                 int64
rangoEdad               object
rangoIngreso            object
flagGenero               int64
flagLimaProvCliente      int64
ubigeoCliente          float64
rangoCtdProdAct         object
rangoCtdProdPas         object
rangoCtdProdSeg         object
flagBxi                  int64
saldoTcEntidad1         object
saldoTcEntidad2         object
saldoTcEntidad3         object
saldoTcEntidad4         object
codGiro                float64
flagLimaProvEstab      float64
ubigeoEstab            float64
dtype: object

In [27]:
TrainTot.drop(['ratingMonto'],axis=1).head()

,codCliente,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab,rangoEdad,rangoIngreso,flagGenero,flagLimaProvCliente,ubigeoCliente,rangoCtdProdAct,rangoCtdProdPas,rangoCtdProdSeg,flagBxi,saldoTcEntidad1,saldoTcEntidad2,saldoTcEntidad3,saldoTcEntidad4
0,7649,138.0,43629,1,176.0,Rango4,Rango2,1,1,281.0,Rango1,Rango2,Rango1,1,Rango3,Rango1,Rango1,Rango1
1,7649,75.0,830,1,157.0,Rango4,Rango2,1,1,281.0,Rango1,Rango2,Rango1,1,Rango3,Rango1,Rango1,Rango1
2,7649,75.0,829,1,175.0,Rango4,Rango2,1,1,281.0,Rango1,Rango2,Rango1,1,Rango3,Rango1,Rango1,Rango1
3,7649,114.0,21603,1,175.0,Rango4,Rango2,1,1,281.0,Rango1,Rango2,Rango1,1,Rango3,Rango1,Rango1,Rango1
4,7649,110.0,46216,1,203.0,Rango4,Rango2,1,1,281.0,Rango1,Rango2,Rango1,1,Rango3,Rango1,Rango1,Rango1


In [28]:
TrainTot.shape

(1591617, 19)

In [29]:
sorted(list(TrainTot.drop(['ratingMonto'],axis=1).columns)) == sorted(list(TesTot.columns))

True

In [39]:
print(dir())

['BasePred', 'BaseSub', 'FeatEstab', 'In', 'Out', 'TesTot', 'TrainTot', '_', '_10', '_11', '_12', '_13', '_14', '_16', '_17', '_18', '_19', '_2', '_20', '_21', '_22', '_23', '_24', '_25', '_26', '_27', '_28', '_29', '_3', '_30', '_4', '_5', '_6', '_7', '_8', '_9', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_i17', '_i18', '_i19', '_i2', '_i20', '_i21', '_i22', '_i23', '_i24', '_i25', '_i26', '_i27', '_i28', '_i29', '_i3', '_i30', '_i31', '_i32', '_i33', '_i34', '_i35', '_i36', '_i37', '_i38', '_i39', '_i4', '_i5', '_i6', '_i7', '_i8', '_i9', '_ih', '_ii', '_iii', '_oh', 'add_col_dates', 'collections', 'combine_client_estab', 'data_perfil', 'data_train', 'exit', 'get_ipython', 'get_memory_usage', 'matplotlib', 'mean_squared_error', 'mt', 'np', 'null_verificator', 'pd', 'plt', 'quit', 'reduce_size_data', 'set_client_estab', 'set_est_client_pred', 's

In [40]:
TesTot.head()

,codCliente,codEstab,rangoEdad,rangoIngreso,flagGenero,flagLimaProvCliente,ubigeoCliente,rangoCtdProdAct,rangoCtdProdPas,rangoCtdProdSeg,flagBxi,saldoTcEntidad1,saldoTcEntidad2,saldoTcEntidad3,saldoTcEntidad4,codGiro,flagLimaProvEstab,ubigeoEstab
0,1182,68644,Rango5,Rango6,0,1,250.0,Rango1,Rango3,Rango1,1,Rango2,Rango2,Rango1,Rango1,49.0,1.0,147.0
1,1279,789,Rango6,Rango6,1,1,280.0,Rango1,Rango1,Rango1,1,Rango1,Rango4,Rango1,Rango1,85.0,1.0,156.0
2,12884,6263,Rango6,Rango1,0,1,275.0,Rango2,Rango2,Rango2,1,Rango1,Rango2,Rango1,Rango1,85.0,1.0,170.0
3,1333,899,Rango6,Rango5,1,0,114.0,Rango1,Rango1,Rango2,1,Rango1,Rango1,Rango2,Rango1,89.0,0.0,14.0
4,1366,6180,Rango6,Rango1,0,1,271.0,Rango1,Rango1,Rango2,0,NaN,NaN,NaN,NaN,75.0,1.0,166.0


In [41]:
TesTot.sort_values(by=['codCliente', 'codEstab']).head(10)

,codCliente,codEstab,rangoEdad,rangoIngreso,flagGenero,flagLimaProvCliente,ubigeoCliente,rangoCtdProdAct,rangoCtdProdPas,rangoCtdProdSeg,flagBxi,saldoTcEntidad1,saldoTcEntidad2,saldoTcEntidad3,saldoTcEntidad4,codGiro,flagLimaProvEstab,ubigeoEstab
28161,1,39,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1,105.0,1.0,147.0
244120,1,3843,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1,50.0,1.0,170.0
40138,1,6314,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1,NaN,0.0,NaN
407005,1,12767,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1,NaN,NaN,NaN
378968,1,26260,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1,85.0,0.0,128.0
269632,1,29255,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1,110.0,1.0,147.0
362028,1,35247,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1,95.0,1.0,136.0
418272,1,36889,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1,110.0,1.0,147.0
206748,1,38534,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1,85.0,1.0,152.0
346411,1,46867,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1,110.0,1.0,147.0


In [43]:
TrainTot.drop(['ratingMonto'], axis=1).sort_values(by=['codCliente', 'codEstab']).head(10)

,codCliente,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab,rangoEdad,rangoIngreso,flagGenero,flagLimaProvCliente,ubigeoCliente,rangoCtdProdAct,rangoCtdProdPas,rangoCtdProdSeg,flagBxi,saldoTcEntidad1,saldoTcEntidad2,saldoTcEntidad3,saldoTcEntidad4
1117172,1,105.0,39,1,147.0,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1
1117183,1,105.0,39,1,147.0,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1
1117187,1,105.0,39,1,147.0,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1
1117197,1,105.0,39,1,147.0,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1
1117189,1,72.0,1194,1,147.0,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1
1117182,1,66.0,1911,1,136.0,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1
1117190,1,NaN,2399,0,NaN,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1
1117199,1,110.0,2425,1,170.0,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1
1117205,1,110.0,2425,1,170.0,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1
1117179,1,50.0,3843,1,170.0,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1


In [44]:
TrainTot.drop(['ratingMonto'], axis=1).shape

(1591617, 18)

In [45]:
#Al eliminar la columna fecha y ctdrx se generar DUPLICADOS EN LA DATA DE TRAIN
#lo cual conlleva  apensar en usar el dato fecha para obtener un valor unico
TrainTot.drop(['ratingMonto'], axis=1).drop_duplicates().shape

(901967, 18)

In [46]:
TrainTest = pd.concat(
    (TrainTot.drop(['ratingMonto'],axis=1),TesTot)
)

In [49]:
TrainTest.shape, TrainTot.drop(['ratingMonto'], axis=1).shape, TesTot.shape

((2058820, 18), (1591617, 18), (467203, 18))

In [50]:
TrainTest.sort_values(by=['codCliente', 'codEstab']).head(10)

,codCliente,codEstab,codGiro,flagBxi,flagGenero,flagLimaProvCliente,flagLimaProvEstab,rangoCtdProdAct,rangoCtdProdPas,rangoCtdProdSeg,rangoEdad,rangoIngreso,saldoTcEntidad1,saldoTcEntidad2,saldoTcEntidad3,saldoTcEntidad4,ubigeoCliente,ubigeoEstab
1117172,1,39,105.0,1,1,1,1.0,Rango1,Rango2,Rango1,Rango6,Rango5,Rango1,Rango1,Rango1,Rango1,246.0,147.0
1117183,1,39,105.0,1,1,1,1.0,Rango1,Rango2,Rango1,Rango6,Rango5,Rango1,Rango1,Rango1,Rango1,246.0,147.0
1117187,1,39,105.0,1,1,1,1.0,Rango1,Rango2,Rango1,Rango6,Rango5,Rango1,Rango1,Rango1,Rango1,246.0,147.0
1117197,1,39,105.0,1,1,1,1.0,Rango1,Rango2,Rango1,Rango6,Rango5,Rango1,Rango1,Rango1,Rango1,246.0,147.0
28161,1,39,105.0,1,1,1,1.0,Rango1,Rango2,Rango1,Rango6,Rango5,Rango1,Rango1,Rango1,Rango1,246.0,147.0
1117189,1,1194,72.0,1,1,1,1.0,Rango1,Rango2,Rango1,Rango6,Rango5,Rango1,Rango1,Rango1,Rango1,246.0,147.0
1117182,1,1911,66.0,1,1,1,1.0,Rango1,Rango2,Rango1,Rango6,Rango5,Rango1,Rango1,Rango1,Rango1,246.0,136.0
1117190,1,2399,NaN,1,1,1,0.0,Rango1,Rango2,Rango1,Rango6,Rango5,Rango1,Rango1,Rango1,Rango1,246.0,NaN
1117199,1,2425,110.0,1,1,1,1.0,Rango1,Rango2,Rango1,Rango6,Rango5,Rango1,Rango1,Rango1,Rango1,246.0,170.0
1117205,1,2425,110.0,1,1,1,1.0,Rango1,Rango2,Rango1,Rango6,Rango5,Rango1,Rango1,Rango1,Rango1,246.0,170.0


In [51]:
TrainTest.dtypes

codCliente               int64
codEstab                 int64
codGiro                float64
flagBxi                  int64
flagGenero               int64
flagLimaProvCliente      int64
flagLimaProvEstab      float64
rangoCtdProdAct         object
rangoCtdProdPas         object
rangoCtdProdSeg         object
rangoEdad               object
rangoIngreso            object
saldoTcEntidad1         object
saldoTcEntidad2         object
saldoTcEntidad3         object
saldoTcEntidad4         object
ubigeoCliente          float64
ubigeoEstab            float64
dtype: object

In [52]:
TrainTest = pd.get_dummies(TrainTest)

In [53]:
TrainTest.columns

Index(['codCliente', 'codEstab', 'codGiro', 'flagBxi', 'flagGenero',
       'flagLimaProvCliente', 'flagLimaProvEstab', 'ubigeoCliente',
       'ubigeoEstab', 'rangoCtdProdAct_Rango1', 'rangoCtdProdAct_Rango2',
       'rangoCtdProdAct_Rango3', 'rangoCtdProdPas_Rango1',
       'rangoCtdProdPas_Rango2', 'rangoCtdProdPas_Rango3',
       'rangoCtdProdSeg_Rango1', 'rangoCtdProdSeg_Rango2',
       'rangoCtdProdSeg_Rango3', 'rangoCtdProdSeg_Rango4', 'rangoEdad_Rango1',
       'rangoEdad_Rango2', 'rangoEdad_Rango3', 'rangoEdad_Rango4',
       'rangoEdad_Rango5', 'rangoEdad_Rango6', 'rangoIngreso_Rango1',
       'rangoIngreso_Rango2', 'rangoIngreso_Rango3', 'rangoIngreso_Rango4',
       'rangoIngreso_Rango5', 'rangoIngreso_Rango6', 'saldoTcEntidad1_Rango1',
       'saldoTcEntidad1_Rango2', 'saldoTcEntidad1_Rango3',
       'saldoTcEntidad1_Rango4', 'saldoTcEntidad1_Rango5',
       'saldoTcEntidad1_Rango6', 'saldoTcEntidad2_Rango1',
       'saldoTcEntidad2_Rango2', 'saldoTcEntidad2_Rango3',
     

In [54]:
TrainTest.isnull().any()

codCliente                False
codEstab                  False
codGiro                    True
flagBxi                   False
flagGenero                False
flagLimaProvCliente       False
flagLimaProvEstab          True
ubigeoCliente              True
ubigeoEstab                True
rangoCtdProdAct_Rango1    False
rangoCtdProdAct_Rango2    False
rangoCtdProdAct_Rango3    False
rangoCtdProdPas_Rango1    False
rangoCtdProdPas_Rango2    False
rangoCtdProdPas_Rango3    False
rangoCtdProdSeg_Rango1    False
rangoCtdProdSeg_Rango2    False
rangoCtdProdSeg_Rango3    False
rangoCtdProdSeg_Rango4    False
rangoEdad_Rango1          False
rangoEdad_Rango2          False
rangoEdad_Rango3          False
rangoEdad_Rango4          False
rangoEdad_Rango5          False
rangoEdad_Rango6          False
rangoIngreso_Rango1       False
rangoIngreso_Rango2       False
rangoIngreso_Rango3       False
rangoIngreso_Rango4       False
rangoIngreso_Rango5       False
rangoIngreso_Rango6       False
saldoTcE

In [56]:
X = TrainTest[:TrainTot.shape[0]]
y = TrainTot.ratingMonto

In [57]:
X.sort_values(by=['codCliente', 'codEstab']).head(10)

,codCliente,codEstab,codGiro,flagBxi,flagGenero,flagLimaProvCliente,flagLimaProvEstab,ubigeoCliente,ubigeoEstab,rangoCtdProdAct_Rango1,rangoCtdProdAct_Rango2,rangoCtdProdAct_Rango3,rangoCtdProdPas_Rango1,rangoCtdProdPas_Rango2,rangoCtdProdPas_Rango3,rangoCtdProdSeg_Rango1,rangoCtdProdSeg_Rango2,rangoCtdProdSeg_Rango3,rangoCtdProdSeg_Rango4,rangoEdad_Rango1,rangoEdad_Rango2,rangoEdad_Rango3,rangoEdad_Rango4,rangoEdad_Rango5,rangoEdad_Rango6,rangoIngreso_Rango1,rangoIngreso_Rango2,rangoIngreso_Rango3,rangoIngreso_Rango4,rangoIngreso_Rango5,rangoIngreso_Rango6,saldoTcEntidad1_Rango1,saldoTcEntidad1_Rango2,saldoTcEntidad1_Rango3,saldoTcEntidad1_Rango4,saldoTcEntidad1_Rango5,saldoTcEntidad1_Rango6,saldoTcEntidad2_Rango1,saldoTcEntidad2_Rango2,saldoTcEntidad2_Rango3,saldoTcEntidad2_Rango4,saldoTcEntidad2_Rango5,saldoTcEntidad3_Rango1,saldoTcEntidad3_Rango2,saldoTcEntidad3_Rango3,saldoTcEntidad3_Rango4,saldoTcEntidad3_Rango5,saldoTcEntidad3_Rango6,saldoTcEntidad4_Rango1,saldoTcEntidad4_Rango2,saldoTcEntidad4_Rango3,saldoTcEntidad4_Rango4,saldoTcEntidad4_Rango5,saldoTcEntidad4_Rango6
1117172,1,39,105.0,1,1,1,1.0,246.0,147.0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
1117183,1,39,105.0,1,1,1,1.0,246.0,147.0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
1117187,1,39,105.0,1,1,1,1.0,246.0,147.0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
1117197,1,39,105.0,1,1,1,1.0,246.0,147.0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
1117189,1,1194,72.0,1,1,1,1.0,246.0,147.0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
1117182,1,1911,66.0,1,1,1,1.0,246.0,136.0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
1117190,1,2399,NaN,1,1,1,0.0,246.0,NaN,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
1117199,1,2425,110.0,1,1,1,1.0,246.0,170.0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
1117205,1,2425,110.0,1,1,1,1.0,246.0,170.0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
1117179,1,3843,50.0,1,1,1,1.0,246.0,170.0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0


In [59]:
X.shape, X.drop_duplicates().shape

((1591617, 54), (901967, 54))